In [1]:
import numpy as np

In [2]:
X_train = np.load('Datasets/Fashion_MNIST/X_train.npy')
X_test = np.load('Datasets/Fashion_MNIST/X_test.npy')
y_train = np.load('Datasets/Fashion_MNIST/y_train.npy')
y_test = np.load('Datasets/Fashion_MNIST/y_test.npy')

In [3]:
X_train.shape

(60000, 784)

In [4]:
y_train.shape

(60000,)

In [5]:
from sklearn.preprocessing import OneHotEncoder

In [6]:
encoder = OneHotEncoder(sparse=False)

In [7]:
encoder.fit(y_train.reshape(-1, 1))

OneHotEncoder(categorical_features='all', dtype=<class 'numpy.float64'>,
       handle_unknown='error', n_values='auto', sparse=False)

In [8]:
y_train = encoder.transform(y_train.reshape(-1, 1))
y_test = encoder.transform(y_test.reshape(-1, 1))

In [9]:
def get_next_batch(X, y, size):
    m = X.shape[0]
    index = np.random.permutation(np.arange(m))
    for i in range(m//size):
        batch_i = index[i*size: i*size+size]
        yield X[batch_i], y[batch_i]

### Training

In [10]:
import tensorflow as tf

C:\ProgramData\Anaconda3\envs\tfgpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [1]:
def dense(X, units, activation=None, kernel_initializer=None, name=None):
    with tf.name_scope(f'dense/{name}'):
        
        if kernel_initializer:
            initializer = kernel_initializer()
        else:
            initializer = tf.random_normal_initializer()
            
        W = tf.Variable(initializer([X.shape[1].value, units]), name='W')
        b = tf.Variable(tf.zeros([units]), name='b')
        Z = tf.add(tf.matmul(X, W), b)
        
        if activation:
            A = activation(Z)
        else:
            A = Z
            
    return A

In [12]:
def dnn_model(X, y, layers, activation, kernel_initializer):
    current_state = X
    for i, layer in enumerate(layers):
        current_state = dense(current_state, layer, activation, kernel_initializer, name=f"layer_{i+1}")
    output = dense(current_state, y.shape[1].value, kernel_initializer=kernel_initializer, name=f"output" )
    return output

In [13]:
def optimize(X_train, y_train, layers, learning_rate, epochs, batch_size, X_test, y_test, print_acc=False):
    
    n_features = X_train.shape[1]
    n_labels = y_train.shape[1]
    
    X = tf.placeholder(tf.float32, shape=[None, n_features], name='features')
    y = tf.placeholder(tf.float32, shape=[None, n_labels], name='labels')
    
    output = dnn_model(X, y, layers, tf.tanh, tf.contrib.layers.xavier_initializer)
    
    correct_pred = tf.equal(tf.argmax(output,1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))
    
    loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y, logits=output)
    cost = tf.reduce_mean(loss)

    train_op = tf.train.AdamOptimizer(learning_rate=LR).minimize(cost)
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        for epoch in range(1, epochs+1):
            for i, (X_batch, y_batch) in enumerate(get_next_batch(X_train, y_train, Batch_size)):
                _, c, train_acc = sess.run([train_op, cost, accuracy], {X: X_batch, y: y_batch})

            if print_acc and epoch%10 == 0:
                test_loss, test_acc = sess.run([cost, accuracy], {X: X_test, y: y_test})
                print("Epoch:", np.round(epoch,4),
                      "Train Loss:", np.round(c, 4),
                      "Train accuracy:", np.round(train_acc, 4),
                      "Test Loss:", np.round(test_loss, 4),
                      "Test accuracy:", np.round(test_acc,4))

In [14]:
Batch_size = 512
LR = 0.0005
epochs = 100

In [15]:
Layers = [256, 128, 64, 32]

In [16]:
tf.reset_default_graph()

In [17]:
optimize(X_train, y_train, Layers, learning_rate=LR, epochs=epochs, batch_size=Batch_size, 
         X_test=X_test, y_test=y_test, print_acc=True)

Epoch: 10 Train Loss: 0.5961 Train accuracy: 0.791 Test Loss: 0.5735 Test accuracy: 0.7911
Epoch: 20 Train Loss: 0.5421 Train accuracy: 0.7969 Test Loss: 0.5689 Test accuracy: 0.7898
Epoch: 30 Train Loss: 0.3898 Train accuracy: 0.8496 Test Loss: 0.5069 Test accuracy: 0.8143
Epoch: 40 Train Loss: 0.4847 Train accuracy: 0.8125 Test Loss: 0.5158 Test accuracy: 0.805
Epoch: 50 Train Loss: 0.4508 Train accuracy: 0.8262 Test Loss: 0.4969 Test accuracy: 0.8112
Epoch: 60 Train Loss: 0.5297 Train accuracy: 0.8027 Test Loss: 0.5029 Test accuracy: 0.8125
Epoch: 70 Train Loss: 0.5385 Train accuracy: 0.8047 Test Loss: 0.5005 Test accuracy: 0.8109
Epoch: 80 Train Loss: 0.4435 Train accuracy: 0.8477 Test Loss: 0.4808 Test accuracy: 0.8211
Epoch: 90 Train Loss: 0.4579 Train accuracy: 0.8281 Test Loss: 0.4643 Test accuracy: 0.8305
Epoch: 100 Train Loss: 0.417 Train accuracy: 0.8242 Test Loss: 0.4469 Test accuracy: 0.8371
